In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vqa-2ansques/v2_mscoco_val2014_annotations.json
/kaggle/input/vqa-2ansques/val_ques.json
/kaggle/input/vqa-2ansques/v2_mscoco_train2014_annotations.json
/kaggle/input/vqa-2ansques/Train_ques.json
/kaggle/input/coco-2014-dataset-for-yolov3/coco2014/5k.part
/kaggle/input/coco-2014-dataset-for-yolov3/coco2014/trainvalno5k.part
/kaggle/input/coco-2014-dataset-for-yolov3/coco2014/coco.names


In [41]:
import os
train_ans_path="/kaggle/input/vqa-2ansques/v2_mscoco_train2014_annotations.json"
train_ques_path="/kaggle/input/vqa-2ansques/Train_ques.json"
val_ans_path="/kaggle/input/vqa-2ansques/v2_mscoco_val2014_annotations.json"
val_ques_path="/kaggle/input/vqa-2ansques/val_ques.json"
coco_path="/kaggle/input/coco-2014-dataset-for-yolov3/coco2014/"

In [42]:
os.listdir(coco_path)

['5k.part',
 'labels',
 'trainvalno5k.part',
 'annotations',
 'images',
 'coco.names']

In [43]:
images_path=os.path.join(coco_path,'images')
os.listdir(images_path)

['val2014', 'test2014', 'train2014']

In [44]:
train_images_dir=os.path.join(images_path,'train2014')
val_images_dir=os.path.join(images_path,'val2014')
print("train_images_dir:", train_images_dir,"\n","val_images_dir:",val_images_dir )

train_images_dir: /kaggle/input/coco-2014-dataset-for-yolov3/coco2014/images/train2014 
 val_images_dir: /kaggle/input/coco-2014-dataset-for-yolov3/coco2014/images/val2014


Checking data inside data set

In [45]:
paths = {
    "train annotations": train_ans_path,
    "train questions": train_ques_path,
    "val annotations": val_ans_path,
    "val questions": val_ques_path,
    "train images directory": train_images_dir,
    "val images directory": val_images_dir
}
for name, path in paths.items():
    if os.path.exists(path):
        print(name, "exists")
    else:
        print(name, "not found")


train annotations exists
train questions exists
val annotations exists
val questions exists
train images directory exists
val images directory exists


In [46]:

# Load json files
import json

with open(train_ques_path, "r") as f:
    train_questions_json = json.load(f)

with open(train_ans_path, "r") as f:
    train_annotations_json = json.load(f)
    
print("Number of train questions:",
      len(train_questions_json["questions"]))

print("Number of train annotations:",
      len(train_annotations_json["annotations"]))

# Count images

train_image_count = len(os.listdir(train_images_dir))
val_image_count = len(os.listdir(val_images_dir))

print("Train images:", train_image_count)
print("Val images:", val_image_count)

Number of train questions: 443757
Number of train annotations: 443757
Train images: 82783
Val images: 40458


***Load JSON and build question_id maps***

In [47]:
# Load json files
import json

with open(train_ques_path, "r") as f:
    train_questions_json = json.load(f)

with open(train_ans_path, "r") as f:
    train_annotations_json = json.load(f)

# question_id to string
train_qid_to_question = {}

for q in train_questions_json["questions"]:
    train_qid_to_question[q["question_id"]] = q["question"]

# Build question id to annotation map
# key: question_id
# value: annotation dict

train_q2ann = {}

for ann in train_annotations_json["annotations"]:
    train_q2ann[ann["question_id"]] = ann

# Sanity check
sample_qid = next(iter(train_qid_to_question))

print("Sample question id:", sample_qid)
print("Question:", train_qid_to_question[sample_qid])
print("Annotation keys:", train_q2ann[sample_qid].keys())

Sample question id: 458752000
Question: What is this photo taken looking through?
Annotation keys: dict_keys(['question_type', 'multiple_choice_answer', 'answers', 'image_id', 'answer_type', 'question_id'])


In [49]:
import os

#image_id to image filename
def get_train_image_path(image_id):
    image_name = f"COCO_train2014_{image_id:012d}.jpg"
    return os.path.join(train_images_dir, image_name)

train_records = []
missing_image_count = 0

for qid, question in train_qid_to_question.items():
    ann = train_q2ann[qid]
    
    image_id = ann["image_id"]
    image_path = get_train_image_path(image_id)
    
    # Check if image exists
    if not os.path.exists(image_path):
        missing_image_count += 1
        continue
    
    answers = [ans["answer"] for ans in ann["answers"]]
    
    record = {
        "image_path": image_path,
        "question": question,
        "answers": answers
    }
    
    train_records.append(record)

print("Total questions processed:", len(train_qid_to_question))
print("Valid training records:", len(train_records))
print("Missing images:", missing_image_count)

print("\nSample record")
print(train_records[0])


Total questions processed: 443757
Valid training records: 443757
Missing images: 0

Sample record
{'image_path': '/kaggle/input/coco-2014-dataset-for-yolov3/coco2014/images/train2014/COCO_train2014_000000458752.jpg', 'question': 'What is this photo taken looking through?', 'answers': ['net', 'net', 'net', 'netting', 'net', 'net', 'mesh', 'net', 'net', 'net']}


In [50]:
# Load validation json files
with open(val_ques_path, "r") as f:
    val_questions_json = json.load(f)

with open(val_ans_path, "r") as f:
    val_annotations_json = json.load(f)

# Build question id maps for validation
val_qid_to_question = {}
for q in val_questions_json["questions"]:
    val_qid_to_question[q["question_id"]] = q["question"]

val_q2ann = {}
for ann in val_annotations_json["annotations"]:
    val_q2ann[ann["question_id"]] = ann

# Function to convert image_id to validation image path
def get_val_image_path(image_id):
    image_name = f"COCO_val2014_{image_id:012d}.jpg"
    return os.path.join(val_images_dir, image_name)

# Build validation records
val_records = []
missing_val_images = 0

for qid, question in val_qid_to_question.items():
    ann = val_q2ann[qid]
    
    image_id = ann["image_id"]
    image_path = get_val_image_path(image_id)
    
    if not os.path.exists(image_path):
        missing_val_images += 1
        continue
    
    answers = [ans["answer"] for ans in ann["answers"]]
    
    record = {
        "image_path": image_path,
        "question": question,
        "answers": answers
    }
    
    val_records.append(record)

print("Total validation questions:", len(val_qid_to_question))
print("Valid validation records:", len(val_records))
print("Missing validation images:", missing_val_images)

print("\nSample validation record")
print(val_records[0])


Total validation questions: 214354
Valid validation records: 214124
Missing validation images: 230

Sample validation record
{'image_path': '/kaggle/input/coco-2014-dataset-for-yolov3/coco2014/images/val2014/COCO_val2014_000000262148.jpg', 'question': 'Where is he looking?', 'answers': ['down', 'down', 'at table', 'skateboard', 'down', 'table', 'down', 'down', 'down', 'down']}


***Answer Vocabulary & Preprocessing***

In [56]:
import re

# Answer preprocessing
def preprocess_answer(ans):
    ans = ans.lower()
    
    # Remove punctuation
    ans = re.sub(r"[^\w\s]", "", ans)
    
    # Normalize numbers
    number_map = {
        "zero": "0",
        "one": "1",
        "two": "2",
        "three": "3",
        "four": "4",
        "five": "5",
        "six": "6",
        "seven": "7",
        "eight": "8",
        "nine": "9",
        "ten": "10"
    }
    
    if ans in number_map:
        ans = number_map[ans]
    
    # Remove extra spaces
    ans = ans.strip()
    
    return ans

# Quick sanity check
print(preprocess_answer("Two"))
print(preprocess_answer("tennis-ball"))
print(preprocess_answer("  Grey "))


2
tennisball
grey


In [57]:
from collections import Counter

# Count all processed answers
answer_counter = Counter()

for record in train_records:
    for ans in record["answers"]:
        processed_ans = preprocess_answer(ans)
        answer_counter[processed_ans] += 1

print("Total unique answers before filtering:", len(answer_counter))

# Show most common answers
for ans, freq in answer_counter.most_common(10):
    print(ans,"->", freq)


Total unique answers before filtering: 157141
no -> 834583
yes -> 825618
2 -> 122862
1 -> 121129
white -> 80731
3 -> 62344
red -> 48293
black -> 47321
blue -> 46528
0 -> 46338


In [59]:
# Select top 3000 answers
TOP_K = 3000

most_common_answers = answer_counter.most_common(TOP_K)

ans2idx = {}
idx2ans = []

for idx, (ans, _) in enumerate(most_common_answers):
    ans2idx[ans] = idx
    idx2ans.append(ans)

print("Final answer vocabulary size:", len(ans2idx))

# Quick check
print("Index of answer 'yes' if present:", ans2idx.get("yes", "not found"))
print("Answer at index 0:", idx2ans[0])


Final answer vocabulary size: 3000
Index of answer 'yes' if present: 1
Answer at index 0: no


In [60]:
import torch

def build_soft_target(answers, ans2idx):
    # Output tensor shape: (3000,)
    target = torch.zeros(len(ans2idx))
    
    for ans in answers:
        processed_ans = preprocess_answer(ans)
        if processed_ans in ans2idx:
            target[ans2idx[processed_ans]] += 1
    
    # Normalize as VQA soft score
    target = torch.clamp(target / 10.0, max=1.0)
    
    return target

# Test on one sample
sample_target = build_soft_target(train_records[0]["answers"], ans2idx)

print("Target tensor shape:", sample_target.shape)
print("Non zero entries:", sample_target.nonzero().shape[0])


Target tensor shape: torch.Size([3000])
Non zero entries: 2


***Dataset and DataLoader***

In [62]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

image_transform = transforms.Compose([
    transforms.Resize((224, 224)), # (3, 224, 224)
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [63]:

# BERT tokenizer for TEXT
from transformers import BertTokenizer

# Input ids shape: (max_len,)
# Attention mask shape: (max_len,)

MAX_QUESTION_LEN = 21

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [64]:
class VQADataset(Dataset):
    def __init__(self, records, ans2idx, tokenizer, transform, max_len):
        self.records = records
        self.ans2idx = ans2idx
        self.tokenizer = tokenizer
        self.transform = transform
        self.max_len = max_len

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        record = self.records[idx]

        # Load image
        image = Image.open(record["image_path"]).convert("RGB")
        image = self.transform(image)
        # image shape: (3, 224, 224)

        # Tokenize question
        encoding = self.tokenizer(
            record["question"],
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )

        input_ids = encoding["input_ids"].squeeze(0)
        # input_ids shape: (max_len,)

        attention_mask = encoding["attention_mask"].squeeze(0)
        # attention_mask shape: (max_len,)

        # Build soft target
        target = build_soft_target(record["answers"], self.ans2idx)
        # target shape: (3000,)

        return image, input_ids, attention_mask, target


In [65]:
# Create datasets
train_dataset = VQADataset(
    records=train_records,
    ans2idx=ans2idx,
    tokenizer=tokenizer,
    transform=image_transform,
    max_len=MAX_QUESTION_LEN
)

val_dataset = VQADataset(
    records=val_records,
    ans2idx=ans2idx,
    tokenizer=tokenizer,
    transform=image_transform,
    max_len=MAX_QUESTION_LEN
)

# DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print("Train batches:", len(train_loader))
print("Val batches:", len(val_loader))


Train batches: 13868
Val batches: 6692


In [66]:
#Quick Check
lengths = []

for rec in train_records[:1000]:
    lengths.append(len(tokenizer.tokenize(rec["question"])))

print("Max length in sample:", max(lengths))
print("Average length:", sum(lengths) / len(lengths))


Max length in sample: 20
Average length: 7.572


***Model***

In [67]:
import torch
import torch.nn as nn
from torchvision.models import resnet50

class ResNetBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        
        resnet = resnet50(pretrained=True)
        
        # Remove avgpool and fc
        self.backbone = nn.Sequential(
            resnet.conv1,
            resnet.bn1,
            resnet.relu,
            resnet.maxpool,
            resnet.layer1,
            resnet.layer2,
            resnet.layer3,
            resnet.layer4
        )

    def forward(self, x):
        # x shape: (batch_size, 3, 224, 224)
        feat = self.backbone(x)
        # feat shape: (batch_size, 2048, 7, 7)
        return feat


In [68]:
class SpatialAttention(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        
        self.conv = nn.Conv2d(
            in_channels=in_channels,
            out_channels=1,
            kernel_size=1
        )

    def forward(self, x):
        # x shape: (batch_size, 2048, 7, 7)
        
        attn_map = self.conv(x)
        # attn_map shape: (batch_size, 1, 7, 7)
        
        attn_map = attn_map.view(attn_map.size(0), -1)
        # shape: (batch_size, 49)
        
        attn_weights = torch.softmax(attn_map, dim=1)
        # shape: (batch_size, 49)
        
        attn_weights = attn_weights.view(x.size(0), 1, 7, 7)
        # shape: (batch_size, 1, 7, 7)
        
        weighted_feat = x * attn_weights
        # shape: (batch_size, 2048, 7, 7)
        
        img_embed = weighted_feat.sum(dim=[2, 3])
        # img_embed shape: (batch_size, 2048)
        
        return img_embed


In [69]:
from transformers import BertModel

class TextEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.bert = BertModel.from_pretrained("bert-base-uncased")

    def forward(self, input_ids, attention_mask):
        # input_ids shape: (batch_size, max_len)
        # attention_mask shape: (batch_size, max_len)
        
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        cls_embed = outputs.last_hidden_state[:, 0, :]
        # cls_embed shape: (batch_size, 768)
        
        return cls_embed


In [70]:
class GatedFusion(nn.Module):
    def __init__(self, img_dim, txt_dim, hidden_dim):
        super().__init__()
        
        self.img_proj = nn.Linear(img_dim, hidden_dim)
        self.txt_proj = nn.Linear(txt_dim, hidden_dim)
        
        self.gate = nn.Linear(hidden_dim * 2, hidden_dim)

    def forward(self, img_feat, txt_feat):
        # img_feat shape: (batch_size, 2048)
        # txt_feat shape: (batch_size, 768)
        
        img_h = self.img_proj(img_feat)
        # shape: (batch_size, hidden_dim)
        
        txt_h = self.txt_proj(txt_feat)
        # shape: (batch_size, hidden_dim)
        
        concat = torch.cat([img_h, txt_h], dim=1)
        # shape: (batch_size, hidden_dim * 2)
        
        gate = torch.sigmoid(self.gate(concat))
        # shape: (batch_size, hidden_dim)
        
        fused = gate * img_h + (1 - gate) * txt_h
        # shape: (batch_size, hidden_dim)
        
        return fused


In [71]:
class VQAModel(nn.Module):
    def __init__(self, num_answers):
        super().__init__()
        
        self.image_encoder = ResNetBackbone()
        self.spatial_attn = SpatialAttention(in_channels=2048)
        self.text_encoder = TextEncoder()
        
        self.fusion = GatedFusion(
            img_dim=2048,
            txt_dim=768,
            hidden_dim=1024
        )
        
        self.classifier = nn.Linear(1024, num_answers)

    def forward(self, images, input_ids, attention_mask):
        # images shape: (batch_size, 3, 224, 224)
        # input_ids shape: (batch_size, max_len)
        # attention_mask shape: (batch_size, max_len)
        
        img_feat_map = self.image_encoder(images)
        # shape: (batch_size, 2048, 7, 7)
        
        img_feat = self.spatial_attn(img_feat_map)
        # shape: (batch_size, 2048)
        
        txt_feat = self.text_encoder(input_ids, attention_mask)
        # shape: (batch_size, 768)
        
        fused = self.fusion(img_feat, txt_feat)
        # shape: (batch_size, 1024)
        
        logits = self.classifier(fused)
        # logits shape: (batch_size, 3000)
        
        return logits


In [72]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VQAModel(num_answers=len(ans2idx))
model = model.to(device)

images, input_ids, attention_mask, targets = next(iter(train_loader))

images = images.to(device)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

with torch.no_grad():
    outputs = model(images, input_ids, attention_mask)

print("Output logits shape:", outputs.shape)


Output logits shape: torch.Size([32, 3000])


In [73]:
def freeze_module(module):
    for param in module.parameters():
        param.requires_grad = False

def unfreeze_module(module):
    for param in module.parameters():
        param.requires_grad = True


In [74]:
# Freeze image encoder and text encoder
freeze_module(model.image_encoder)
freeze_module(model.text_encoder)

print("Image encoder frozen")
print("Text encoder frozen")


Image encoder frozen
Text encoder frozen


In [75]:
import torch.optim as optim

# Loss function
# Expects raw logits and soft targets
criterion = nn.BCEWithLogitsLoss()

# Optimizer only sees trainable parameters
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4
)


***Traning Loop***

In [76]:
def train_one_epoch(model, dataloader, optimizer, criterion, device, epoch, phase):
    model.train()
    
    total_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (images, input_ids, attention_mask, targets) in enumerate(dataloader):
        images = images.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images, input_ids, attention_mask)
        # outputs shape: (batch_size, 3000)
        
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Accuracy calculation
        preds = outputs.argmax(dim=1)
        gt = targets.argmax(dim=1)
        
        correct += (preds == gt).sum().item()
        total += targets.size(0)
        
        # Print batch progress every 200 batches
        if batch_idx % 200 == 0:
            print(
                f"{phase} Epoch {epoch} | "
                f"Batch {batch_idx}/{len(dataloader)} | "
                f"Loss {loss.item():.4f}"
            )
    
    avg_loss = total_loss / len(dataloader)
    acc = correct / total
    
    return avg_loss, acc


In [77]:
def validate_one_epoch(model, dataloader, criterion, device):
    model.eval()
    
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, input_ids, attention_mask, targets in dataloader:
            images = images.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            targets = targets.to(device)
            
            outputs = model(images, input_ids, attention_mask)
            loss = criterion(outputs, targets)
            
            total_loss += loss.item()
            
            preds = outputs.argmax(dim=1)
            gt = targets.argmax(dim=1)
            
            correct += (preds == gt).sum().item()
            total += targets.size(0)
    
    avg_loss = total_loss / len(dataloader)
    acc = correct / total
    
    return avg_loss, acc


In [78]:
def save_checkpoint(model, optimizer, epoch, path):
    checkpoint = {
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict()
    }
    torch.save(checkpoint, path)

def load_checkpoint(model, optimizer, path, device):
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint["model_state"])
    optimizer.load_state_dict(checkpoint["optimizer_state"])
    start_epoch = checkpoint["epoch"] + 1
    return start_epoch


In [79]:
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

In [ ]:
num_epochs_frozen = 3
num_epochs_unfrozen = 2

checkpoint_path = "/kaggle/working/vqa_checkpoint.pth"

start_epoch = 0

for epoch in range(start_epoch, num_epochs_frozen):
    train_loss, train_acc = train_one_epoch(
        model=model,
        dataloader=train_loader,
        optimizer=optimizer,
        criterion=criterion,
        device=device,
        epoch=epoch,
        phase="Frozen"
    )
    
    val_loss, val_acc = validate_one_epoch(
        model=model,
        dataloader=val_loader,
        criterion=criterion,
        device=device
    )
    
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)
    val_loss_history.append(val_loss)
    val_acc_history.append(val_acc)
    
    print(
        f"Frozen Epoch {epoch} | "
        f"Train Loss {train_loss:.4f} | Train Acc {train_acc:.4f} | "
        f"Val Loss {val_loss:.4f} | Val Acc {val_acc:.4f}"
        
    )
    
    save_checkpoint(
        model=model,
        optimizer=optimizer,
        epoch=epoch,
        path=checkpoint_path
    )


Frozen Epoch 0 | Batch 0/13868 | Loss 0.0020
Frozen Epoch 0 | Batch 200/13868 | Loss 0.0024
Frozen Epoch 0 | Batch 400/13868 | Loss 0.0016
Frozen Epoch 0 | Batch 600/13868 | Loss 0.0016
Frozen Epoch 0 | Batch 800/13868 | Loss 0.0019
Frozen Epoch 0 | Batch 1000/13868 | Loss 0.0015
Frozen Epoch 0 | Batch 1200/13868 | Loss 0.0017
Frozen Epoch 0 | Batch 1400/13868 | Loss 0.0015
Frozen Epoch 0 | Batch 1600/13868 | Loss 0.0017
Frozen Epoch 0 | Batch 1800/13868 | Loss 0.0014
Frozen Epoch 0 | Batch 2000/13868 | Loss 0.0013
Frozen Epoch 0 | Batch 2200/13868 | Loss 0.0014
Frozen Epoch 0 | Batch 2400/13868 | Loss 0.0014
Frozen Epoch 0 | Batch 2600/13868 | Loss 0.0015
Frozen Epoch 0 | Batch 2800/13868 | Loss 0.0011
Frozen Epoch 0 | Batch 3000/13868 | Loss 0.0011
Frozen Epoch 0 | Batch 3200/13868 | Loss 0.0014
Frozen Epoch 0 | Batch 3400/13868 | Loss 0.0012
Frozen Epoch 0 | Batch 3600/13868 | Loss 0.0013
Frozen Epoch 0 | Batch 3800/13868 | Loss 0.0012
Frozen Epoch 0 | Batch 4000/13868 | Loss 0.0012

In [ ]:
unfreeze_module(model.image_encoder)
unfreeze_module(model.text_encoder)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-5
)

print("Encoders unfrozen and optimizer reset")


In [ ]:
for epoch in range(num_epochs_frozen, num_epochs_frozen + num_epochs_unfrozen):
    train_loss, train_acc = train_one_epoch(
        model=model,
        dataloader=train_loader,
        optimizer=optimizer,
        criterion=criterion,
        device=device,
        epoch=epoch,
        phase="Unfrozen"
    )
    
    val_loss, val_acc = validate_one_epoch(
        model=model,
        dataloader=val_loader,
        criterion=criterion,
        device=device
    )
    
    # Save metrics
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)
    val_loss_history.append(val_loss)
    val_acc_history.append(val_acc)
    
    print(
        f"Unfrozen Epoch {epoch} | "
        f"Train Loss {train_loss:.4f} | Train Acc {train_acc:.4f} | "
        f"Val Loss {val_loss:.4f} | Val Acc {val_acc:.4f}"
    )
    
    save_checkpoint(
        model=model,
        optimizer=optimizer,
        epoch=epoch,
        path=checkpoint_path
    )


***For Saving Model***

In [ ]:
final_model_path = "/kaggle/working/vqa_final_model.pth"

torch.save({
    "model_state": model.state_dict(),
    "ans2idx": ans2idx,
    "idx2ans": idx2ans,
    "max_question_len": MAX_QUESTION_LEN
}, final_model_path)

print("Final trained model saved at:", final_model_path)
